In [12]:
import numpy as np 
import pandas as pd 

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [13]:
train = pd.read_csv("../input/creditcard.csv")
train.info()
X = train.drop("Class", 1)
Y = train["Class"]

In [14]:
train.head(5)

In [15]:
from sklearn.preprocessing import StandardScaler
X['Amount_n']= StandardScaler().fit_transform(X['Amount'].reshape(-1,1))

In [16]:
from sklearn.preprocessing import StandardScaler
X['Amount']= StandardScaler().fit_transform(X['Amount'].reshape(-1,1))
X.head(3)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [18]:
print(len(y_train[y_train==1]), len(y_train[y_train==0]))

In [19]:
from imblearn.over_sampling import SMOTE
oversampler = SMOTE(random_state=0)
X_train ,y_train = oversampler.fit_sample(X_train, y_train)

In [20]:
print(len(y_train[y_train==1]), len(y_train[y_train==0]))

In [21]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [22]:
favorite_clf = RandomForestClassifier()
favorite_clf.fit(X_train, y_train)
prediction = favorite_clf.predict(X_test)

In [23]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=favorite_clf,
                        X=X_train,
                        y=y_train,
                        cv=10,
                        n_jobs=1)

print('Cross validation scores: %s' % scores)

import matplotlib.pyplot as plt
plt.title('Cross validation scores')
plt.scatter(np.arange(len(scores)), scores)
plt.axhline(y=np.mean(scores), color='g') # Mean value of cross validation scores
plt.show()

In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,prediction)